In [1]:
import pandas as pd
import json
from tqdm import tqdm
import csv
import numpy as np
import random

In [2]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def clean_stopword(article):

    for word in STOPWORDS:
        token = ' ' + word + ' '
        article = article.replace(token, ' ')
        article = article.replace(' ', ' ')
    return article

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wuguanwei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_data_train = pd.read_pickle('HW_train.pkl')
df_data_test = pd.read_pickle('HW_test.pkl')

In [5]:
df_data_train['new_text'] = df_data_train['text'].apply(lambda x : clean_stopword(x))

In [6]:
df_data_test['new_text'] = df_data_test['text'].apply(lambda x : clean_stopword(x))

In [7]:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(df_data_train['new_text'])

TfidfVectorizer()

In [8]:
X_train = tfidf_vectorizer.transform(df_data_train['new_text'])
y_train = df_data_train['label']

X_test = tfidf_vectorizer.transform(df_data_test['new_text'])

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
mnb_model = MultinomialNB()
mnb_model = mnb_model.fit(X_train, y_train)

y_train_pred_mnb = mnb_model.predict(X_train)
y_test_pred_mnb = mnb_model.predict(X_test)

acc_train_mnb = accuracy_score(y_true=y_train, y_pred=y_train_pred_mnb)


print('training accuracy: {}'.format(round(acc_train_mnb, 2)))


training accuracy: 0.52


In [10]:
df_data_test['label'] = y_test_pred_mnb

In [11]:
df_data_test

,_score,hashTag,tweet_id,text,new_text,label
2,232,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...","Confident obedience, I write you, knowing even...",anticipation
4,989,[],0x2de201,"""Trust is not the same as faith. A friend is s...","""Trust faith. A friend someone trust. Putting ...",anticipation
9,66,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,When enough ? When satisfied ? Is goal really ...,joy
30,104,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...","God woke up, chase day #GodsPlan #GodsWork <LH>",joy
33,310,[],0x26289a,"In these tough times, who do YOU turn to as yo...","In tough times, YOU turn symbol hope? <LH>",joy
...,...,...,...,...,...,...
1867525,602,[],0x2913b4,"""For this is the message that ye heard from th...","""For message ye heard beginning, love one anot...",anticipation
1867529,598,[],0x2a980e,"""There is a lad here, which hath five barley l...","""There lad here, hath five barley loaves, two ...",anticipation
1867530,827,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,When buy last 2 tickets remaining show sell ou...,joy
1867531,368,[],0x29d0cb,I swear all this hard work gone pay off one da...,I swear hard work gone pay one day😈💰💸 <LH>,joy


In [13]:
submission = df_data_test.drop(['_score','hashTag','text','new_text'],axis = 1)
submission.columns = ['id','emotion']

In [14]:
submission.to_csv('submission.csv',index=False)